In [ ]:
# denormalized_df.show(truncate=False)

In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=1236cb1777fecc28b7fdcfbb385de9cab3429e4efa8304adc50cd7acbe055374
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from google.colab import drive
drive.mount('/content/drive')
from pyspark.sql import SparkSession

Mounted at /content/drive


In [3]:
spark = SparkSession.builder \
    .appName("tax_product") \
    .getOrCreate()

In [22]:
customer_df = spark.read.csv("/content/drive/MyDrive/1_Deloitte_Assignment/customer_table_2", header=True, inferSchema=True)
transaction_df = spark.read.csv("/content/drive/MyDrive/1_Deloitte_Assignment/transaction_table_2", header=True, inferSchema=True)
product_df = spark.read.csv("/content/drive/MyDrive/1_Deloitte_Assignment/product_table_2", header=True, inferSchema=True)
promotion_df = spark.read.csv("/content/drive/MyDrive/1_Deloitte_Assignment/promotion_table_2", header=True, inferSchema=True)
promotion_df = promotion_df.withColumnRenamed("description", "prom_description")

In [23]:
# promotion_df.show()

In [24]:
# ###### denormalized customer #######
# from pyspark.sql.functions import split, explode, regexp_replace

# # Remove brackets from the elements in the purchase_history array column
# customer_df = customer_df.withColumn("purchase_history_array", split(regexp_replace(customer_df["purchase_history"], "[\[\]]", ""), ", "))

# # Explode the purchase_history_array column to create separate rows for each element
# customer_denormalized_df = customer_df.select("customer_id","name","membership_level","geographical_region", explode("purchase_history_array").alias("individual_purchase_history"))

# # Show the denormalized DataFrame
# # customer_denormalized_df.show(truncate=False)


In [25]:
customer_df.show(truncate = False)

+------------------------------------+-----------------+----------------+-------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|customer_id                         |name             |membership_level|geographical_region|purchase_history                                                                                                                                                                                                                                                                                                                                                                                                |
+---------

In [26]:
product_df.show(truncate = False)

+------------------------------------+-----------+------------------------------------+--------+
|product_id                          |description|attributes                          |tax_rate|
+------------------------------------+-----------+------------------------------------+--------+
|58d20dcb-b2f8-4e79-aea4-8fb6b818af94|Printer    |{'color': 'black', 'size': 'medium'}|0.1     |
|03e17f8e-2bb9-4fa3-93d0-013253e8637e|Monitor    |{'color': 'black', 'size': 'medium'}|0.1     |
|112b2485-fc04-45cc-b6c0-4533810dc2e0|Laptop     |{'color': 'black', 'size': 'medium'}|0.07    |
|fd0e5773-8f7f-4304-b062-77ee4e0374f0|Printer    |{'color': 'white', 'size': 'small'} |0.07    |
|7eeb70cf-dc59-4d41-a96c-c821ba7c4b60|Camera     |{'color': 'black', 'size': 'medium'}|0.1     |
|026251ac-8973-4939-a7bf-e1f5c2798fdf|Camera     |{'color': 'red', 'size': 'large'}   |0.07    |
|3108a79f-477f-4ca9-abb7-dc8b4360f76c|Headphones |{'color': 'blue', 'size': 'medium'} |0.1     |
|9b43f55f-7372-4c77-bcce-8834a

In [27]:
###### denormalized promotion #######
from pyspark.sql.functions import split, explode, regexp_replace

# Assuming customer_df is your DataFrame
# Remove brackets from the elements in the purchase_history array column
promotion_df = promotion_df.withColumn("applicable_products_array", split(regexp_replace(promotion_df["applicable_products"], "[\[\]]", ""), ", "))

# Explode the purchase_history_array column to create separate rows for each element
promotion_denormalized_df = promotion_df.select("promo_id","prom_description","discount", explode("applicable_products_array").alias("individual_applicable_product"))

# Show the denormalized DataFrame
# promotion_denormalized_df.show(truncate=False)
promotion_denormalized_df = promotion_denormalized_df.dropDuplicates(['individual_applicable_product'])
promotion_denormalized_df.show(truncate=False)

+------------------------------------+------------------------------------------------------------+--------+------------------------------------+
|promo_id                            |prom_description                                            |discount|individual_applicable_product       |
+------------------------------------+------------------------------------------------------------+--------+------------------------------------+
|f9852fe7-092b-4a07-afb9-197991315f39|Give air smile.                                             |7.31    |00a96e1d-58c8-4a02-8abb-edc2fad7d0a0|
|962a794f-1036-401c-a81b-6885af52f498|Probably Democrat majority floor possible that crime reason.|22.03   |00b79ea8-4b1a-4b96-8faf-d861f21e44e4|
|022ea156-0b0e-4b36-984f-fb9715cc5f99|Allow employee official two focus go.                       |33.92   |015975f4-920a-460d-94ac-b6982de9e346|
|d5a6488d-50a0-4c87-ab33-08a4f47f4f3a|Into watch animal along.                                    |37.69   |02075a09-a017-40

In [28]:
transaction_df.show(truncate=False)

+------------------------------------+------------------------------------+----------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|transaction_id                      |customer_id                         |date      |items                                                                                                                                                                                                                                                                   |
+------------------------------------+------------------------------------+----------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#Data Modelling

###step:-1 Merging customer and transaction table

In [29]:
cust_trans_df = customer_df.join(transaction_df, customer_df["customer_id"] == transaction_df["customer_id"], "left") \
            .select(customer_df["customer_id"], customer_df["name"], customer_df["membership_level"], customer_df["geographical_region"], customer_df["purchase_history"],
                   transaction_df['transaction_id'],transaction_df['date'],transaction_df['items'])

cust_trans_df.show(truncate=False),cust_trans_df.count()

+------------------------------------+-----------------+----------------+-------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|customer_id                         |name             |membership_level|geographical_region|purchase_history                                                                                                                                                                    

(None, 3052)

In [30]:
from pyspark.sql.functions import explode, udf , when, col
from pyspark.sql.types import ArrayType, StringType , MapType
from ast import literal_eval

"""The code is splittnig the itmes meta data into columns for ousr usecase"""

cust_trans_df = cust_trans_df.withColumn("item_purchased",
                                        explode(
                                            udf(lambda x: literal_eval(x) if x is not None else [], ArrayType(StringType()))
                                            (cust_trans_df["items"]))
                                       )
cust_trans_df.show(truncate=False)

+------------------------------------+---------------+----------------+-------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------+
|customer_id                         |name           |membership_level|geographical_region|purchase_history                                                                                                                                                                                        |transaction_id             

In [33]:
import re
from pyspark.sql.types import StringType, MapType

# Define a UDF to parse the nested record string into a dictionary
def parse_nested_record(record):
    if record:
        record_dict = dict(re.findall(r'(\w+)=([\w\.-]+)', record))
        return record_dict
    else:
        return None

parse_nested_record_udf = udf(parse_nested_record, MapType(StringType(), StringType()))

# Apply the UDF to convert the nested record string to a dictionary
cust_trans_df = cust_trans_df.withColumn("record_map", parse_nested_record_udf(col("item_purchased")))
cust_trans_df = cust_trans_df.withColumn("product_id", col("record_map").getItem("product_id"))
cust_trans_df = cust_trans_df.withColumn("price", col("record_map").getItem("price"))
cust_trans_df = cust_trans_df.withColumn("quantity", col("record_map").getItem("quantity"))

cust_trans_df.show(truncate=False)

+------------------------------------+---------------+----------------+-------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+----------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------+------------------------------------------------------------------------------------+------------------------------------+------+--------+
|customer_id                         |name           |membership_level|geographical_region|purchase_history                                                                          

###step - 2 Merging customer, transaction and product table

In [34]:
cust_trans_product_df = cust_trans_df.join(product_df, cust_trans_df["product_id"] == product_df["product_id"], "left") \
                                      .select(cust_trans_df["customer_id"],cust_trans_df["name"],cust_trans_df["membership_level"],
                                              cust_trans_df["geographical_region"],cust_trans_df["purchase_history"],
                                              cust_trans_df["transaction_id"],cust_trans_df["customer_id"],cust_trans_df["date"],cust_trans_df["product_id"],
                                              cust_trans_df["price"],cust_trans_df['quantity'],product_df["description"],product_df["attributes"],product_df["tax_rate"])

cust_trans_product_df.show(truncate=False)

+------------------------------------+---------------+----------------+-------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+------------------------------------+----------+------------------------------------+------+--------+-----------+------------------------------------+--------+
|customer_id                         |name           |membership_level|geographical_region|purchase_history                                                                                                                                                                                        |transaction_id                      |customer_id                         |date      |product_id                          |price |quantity|description|attributes                          |tax_rate|
+---------------------

In [35]:
### selecting appropriate columns for Analysis
cust_trans_product_prom_df = cust_trans_product_df.select("customer_id","name","membership_level","geographical_region","purchase_history","transaction_id","customer_id","date","product_id","price",'quantity',"description","attributes","tax_rate")

###step - 3 Merge all cutomer, transaction, product and promotion table and complete the data modelling part and make data ready for analysis

In [58]:
cust_trans_product_prom_df = cust_trans_product_df.join(promotion_denormalized_df, cust_trans_product_df["product_id"] == promotion_denormalized_df["individual_applicable_product"], "left")
cust_trans_product_prom_df.show(truncate=False)

+------------------------------------+---------------+----------------+-------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+------------------------------------+----------+------------------------------------+------+--------+-----------+------------------------------------+--------+------------------------------------+-------------------------------------------------------+--------+------------------------------------+
|customer_id                         |name           |membership_level|geographical_region|purchase_history                                                                                                                                                                                        |transaction_id                      |customer_id                         |date  

In [36]:
# checking for NULL
# from pyspark.sql.functions import col, sum as sum_
# cust_trans_product_prom_df.select(sum_(col('individual_applicable_product').isNull().cast("int"))).show()

In [37]:
####changing the data type for aggregation
cust_trans_product_prom_df = cust_trans_product_prom_df.withColumn("price", col("price").cast("double"))
cust_trans_product_prom_df.dtypes

[('customer_id', 'string'),
 ('name', 'string'),
 ('membership_level', 'string'),
 ('geographical_region', 'string'),
 ('purchase_history', 'string'),
 ('transaction_id', 'string'),
 ('customer_id', 'string'),
 ('date', 'date'),
 ('product_id', 'string'),
 ('price', 'double'),
 ('quantity', 'string'),
 ('description', 'string'),
 ('attributes', 'string'),
 ('tax_rate', 'double')]

#Analysis

In [38]:
##########Library#############
from pyspark.sql.functions import sum, col


In [48]:
##### 1.1) Total sales per transaction
total_sales_per_transaction = cust_trans_product_prom_df.groupBy('transaction_id').agg(sum('price').alias('Total_sales')).orderBy(col("Total_sales").desc())
total_sales_per_transaction.show(truncate=False),total_sales_per_transaction.count()

+------------------------------------+------------------+
|transaction_id                      |Total_sales       |
+------------------------------------+------------------+
|ad94221f-8680-49f4-b024-fccd615e0c09|1441.0900000000001|
|fd4a1408-a46d-415a-a193-f92a1228149e|1380.7            |
|8b434da8-56e0-4edc-9b7d-e7c36eb8d1dd|1377.54           |
|6a716822-9ad6-4e8b-9375-8547dfe153d5|1365.75           |
|8b674f84-899c-4d56-a204-86c85788a104|1330.68           |
|1beb58ac-eba4-4b94-8b71-aa3ba1240a0a|1330.31           |
|b196000b-f9af-48ef-96b4-dcbb1849e7e1|1325.62           |
|815013d7-063b-4648-8cb3-752c60d5caab|1320.64           |
|1997cff0-3bc0-4729-a6e7-30d08bd36f65|1314.46           |
|697e32b4-a91d-442f-98c3-a51a8044aac8|1310.54           |
|6482e81a-e2e5-4099-b1fe-8ade8c3cf636|1309.24           |
|e7709ebe-e63e-47f8-aca4-fba16dece567|1304.88           |
|b16b9a6e-711f-4930-8944-63e7d3a8f05a|1294.67           |
|fe03adf7-5289-487c-b375-6597a5b4059d|1285.46           |
|dce6cd32-78fd

(None, 3000)

In [49]:
#### 1.2) calculating total price for items
total_price_per_item = cust_trans_product_prom_df.groupBy('description').agg(sum('price').alias('Total_sales'))
total_price_per_item.show(truncate=False),total_price_per_item.count()


+-----------+------------------+
|description|Total_sales       |
+-----------+------------------+
|Smartwatch |167287.14999999994|
|Laptop     |130587.39999999998|
|Mouse      |134075.78000000006|
|Camera     |166177.71         |
|Tablet     |183921.9699999999 |
|Printer    |155836.33         |
|Keyboard   |121635.02000000012|
|Smartphone |161504.53999999992|
|Monitor    |154500.2199999998 |
|Headphones |172990.33000000005|
+-----------+------------------+



(None, 10)

In [50]:
#### 2) Tax calculation by transaction
tax_by_transation = cust_trans_product_prom_df.groupBy('transaction_id', 'description').agg(sum('tax_rate').alias('Total_tax'))
tax_by_transation.show(truncate=False),tax_by_transation.count()

+------------------------------------+-----------+---------+
|transaction_id                      |description|Total_tax|
+------------------------------------+-----------+---------+
|da1cf943-f2bd-41cb-987b-602c21661d64|Camera     |0.2      |
|efa1a5d9-6475-42f4-94b0-6e7a3f1a69f5|Headphones |0.07     |
|b960b06a-9efe-469a-8d60-c4e818d8bfec|Laptop     |0.1      |
|346dde3d-3c1e-40cb-a846-7bee30c26f51|Monitor    |0.15     |
|f8459370-c3ab-4790-906c-041b9ba385cf|Camera     |0.07     |
|cd4bd342-ac79-4461-bde7-efc29d1fed7c|Smartwatch |0.2      |
|b772abb7-8200-4dce-a2ad-b521426428be|Camera     |0.2      |
|9e6d845b-0d2a-4567-9ae2-ed524505fbc6|Keyboard   |0.07     |
|d211aba7-fea4-4e53-b2b4-63a809630850|Mouse      |0.07     |
|eeab1412-b8ef-4672-82ab-d33bbd645745|Smartwatch |0.35     |
|50aa2d8a-1298-4d87-ab0f-ede646b3e519|Smartphone |0.1      |
|9ac35b7b-0fe1-4e3a-9b88-59e1729718b7|Smartphone |0.1      |
|26db6568-bbd0-4eae-8e0a-bce8c080e349|Printer    |0.1      |
|b941287d-85a7-4312-925f

(None, 5654)

In [60]:
############# Data frame updation due to day extraction #############
####### 3) Day wise tax and sales collection
from pyspark.sql.functions import dayofweek, month, year, date_format

cust_trans_product_prom_df = cust_trans_product_prom_df.withColumn("day_name", date_format("date", "EEEE")) \
                                                         .withColumn("month", month("date")) \
                                                         .withColumn("year", year("date"))

day_level_sales_tax = cust_trans_product_prom_df.groupBy('day_name').agg(sum('price').alias('Total_sales'),sum('tax_rate').alias('Total_tax')).orderBy(col("Total_sales").desc(), col("Total_tax"))
day_level_sales_tax.show()

+---------+------------------+------------------+
| day_name|       Total_sales|         Total_tax|
+---------+------------------+------------------+
| Saturday| 242700.3800000003|116.79999999999967|
|  Tuesday|238988.73000000027|115.47999999999959|
| Thursday|231102.13999999998|112.00999999999952|
|   Friday|220229.54000000018|111.78999999999989|
|   Sunday|220060.66000000003|109.02999999999969|
|   Monday|198095.16999999995| 96.47999999999968|
|Wednesday| 197339.8299999999|  94.6399999999998|
+---------+------------------+------------------+



In [52]:
######## 4) total amount spend by each customer over time
amount_spent_by_time = cust_trans_product_prom_df.groupBy('customer_id','date').agg(sum('price').alias('Total_amount')).orderBy(col("Total_amount").desc())
amount_spent_by_time.show(truncate = False)

+------------------------------------+----------+------------------+
|customer_id                         |date      |Total_amount      |
+------------------------------------+----------+------------------+
|eaa9ebe5-453b-4065-970c-e233b866bbcb|2023-03-14|1973.38           |
|d0e207f2-85be-4648-bf30-7649d3e69659|2023-02-23|1441.0900000000001|
|d9057db6-e3ce-4794-a290-cf2a3e2e2c1c|2023-03-10|1380.7            |
|b4a69b2e-8085-44cb-8d21-131882bf2f98|2023-12-11|1377.54           |
|1baf9b9b-2bdc-435b-a3b3-cc063b7cca93|2023-11-17|1365.75           |
|e17ba4ab-501b-4c82-899f-586f5b8c6604|2023-08-08|1330.68           |
|8025e5f6-c1bc-474d-9564-1f0cdb66fc21|2023-01-12|1330.31           |
|9cef33be-712b-41f4-b24f-0f5835aec941|2023-04-18|1325.62           |
|0e7cde04-97be-48a6-a0c0-65a9bc9d767d|2023-05-21|1320.64           |
|1e064033-a04e-4d45-ad3f-b638c858036a|2023-10-10|1314.46           |
|ddce8d76-6739-4894-bbeb-86637219a160|2023-04-03|1310.54           |
|c917621a-d821-49c1-9862-de686b93f

In [53]:
######### 5) product performance metrics
product_performance_metrics = cust_trans_product_prom_df.groupBy('description').agg(sum('price').alias('Total_sales'),sum('tax_rate').alias('Total_tax')).orderBy(col("Total_sales").desc(), col("Total_tax"))
product_performance_metrics.show()

+-----------+------------------+-----------------+
|description|       Total_sales|        Total_tax|
+-----------+------------------+-----------------+
|     Tablet| 183921.9699999999|88.05999999999982|
| Headphones|172990.33000000005|83.58000000000006|
| Smartwatch|167287.14999999994|84.59000000000003|
|     Camera|         166177.71|79.94000000000001|
| Smartphone|161504.53999999992| 75.6900000000001|
|    Printer|         155836.33|            70.35|
|    Monitor| 154500.2199999998|86.69000000000025|
|      Mouse|134075.78000000006|67.26000000000023|
|     Laptop|130587.39999999998|61.95000000000026|
|   Keyboard|121635.02000000012|58.12000000000021|
+-----------+------------------+-----------------+



In [54]:
######## 6) geographical sales distribution
geographical_sales_distribution = cust_trans_product_prom_df.groupBy('geographical_region').agg(sum('price').alias('Total_sales'),sum('tax_rate').alias('Total_tax')).orderBy(col("Total_sales").desc(), col("Total_tax"))
geographical_sales_distribution.show()

+-------------------+------------------+------------------+
|geographical_region|       Total_sales|         Total_tax|
+-------------------+------------------+------------------+
|          Australia|429064.55000000005|209.83999999999637|
|      North America| 407675.9999999993|198.82999999999686|
|               Asia| 392061.9300000002|194.75999999999672|
|             Europe|319713.96999999933|152.79999999999822|
+-------------------+------------------+------------------+



In [56]:
cust_trans_product_prom_df.show()

+--------------------+---------------+----------------+-------------------+--------------------+--------------------+--------------------+----------+--------------------+------+--------+-----------+--------------------+--------+---------+-----+----+
|         customer_id|           name|membership_level|geographical_region|    purchase_history|      transaction_id|         customer_id|      date|          product_id| price|quantity|description|          attributes|tax_rate| day_name|month|year|
+--------------------+---------------+----------------+-------------------+--------------------+--------------------+--------------------+----------+--------------------+------+--------+-----------+--------------------+--------+---------+-----+----+
|9742cbb9-c28e-47d...|Francisco Patel|            Gold|      North America|['d5dc3b47-3242-4...|dbc5a987-9746-41f...|9742cbb9-c28e-47d...|2023-08-23|a6f80668-73a3-4f6...|472.44|       3|      Mouse|{'color': 'red', ...|    0.15|Wednesday|    8|2023|


In [61]:
####### 7) promotion impact analysis
promtion_impact_raw = cust_trans_product_prom_df.select('description','price','discount','tax_rate','day_name','month')
promtion_impact_raw.show()

+-----------+------+--------+--------+---------+-----+
|description| price|discount|tax_rate| day_name|month|
+-----------+------+--------+--------+---------+-----+
|      Mouse|472.44|    NULL|    0.15|Wednesday|    8|
| Smartphone|227.19|    NULL|     0.2|Wednesday|    8|
|    Monitor| 33.44|   47.19|     0.2|  Tuesday|    9|
|     Camera| 420.0|   17.79|     0.1| Thursday|    3|
|    Monitor| 366.5|   49.08|    0.07| Thursday|    3|
|    Monitor| 29.35|   40.65|    0.15|   Friday|   11|
| Smartwatch|381.72|   33.44|    0.15|   Friday|   11|
| Smartphone|133.59|    NULL|    0.15|   Friday|   11|
|    Printer|315.17|     5.4|    0.07| Thursday|    8|
| Headphones|280.47|   40.65|     0.1| Thursday|    8|
|    Monitor|306.79|   38.76|     0.2|   Friday|    6|
|     Tablet|150.61|    6.28|     0.1|   Friday|    4|
| Smartwatch|  39.4|    NULL|    0.15|Wednesday|    8|
|      Mouse|334.77|    NULL|    0.07|  Tuesday|   10|
|    Printer| 84.91|    6.85|     0.1|   Monday|   10|
|     Lapt

In [62]:
# promtion_impact_raw.select('discount').isnull()
############ spliting the data based on the discount applied and no discount applied
withtout_disc = promtion_impact_raw.filter(promtion_impact_raw['discount'].isNull())
with_disc = promtion_impact_raw.filter(promtion_impact_raw['discount'].isNotNull())


In [63]:
promotion_impact_on_item = with_disc.groupBy('description').agg(sum('price').alias('Total_sales'),sum('tax_rate').alias('Total_tax')).orderBy(col("Total_sales").desc(), col("Total_tax"))
no_promotion_impact_on_item = withtout_disc.groupBy('description').agg(sum('price').alias('Total_sales'),sum('tax_rate').alias('Total_tax')).orderBy(col("Total_sales").desc(), col("Total_tax"))

promotion_impact_on_item.show(),no_promotion_impact_on_item.show()


+-----------+------------------+------------------+
|description|       Total_sales|         Total_tax|
+-----------+------------------+------------------+
|    Monitor|127274.81000000003| 73.64000000000019|
|     Tablet|117715.95000000003| 59.08000000000019|
| Smartphone|114783.05999999991| 49.17000000000007|
|     Camera|114471.30999999997| 55.30000000000006|
|    Printer|113944.96999999999| 51.14000000000016|
| Headphones|107429.41999999997| 50.70000000000014|
|   Keyboard|105559.90000000008|52.620000000000175|
| Smartwatch|104978.12999999999| 53.40000000000011|
|     Laptop| 90944.04999999997|41.180000000000106|
|      Mouse| 88141.67000000006|42.030000000000086|
+-----------+------------------+------------------+

+-----------+------------------+------------------+
|description|       Total_sales|         Total_tax|
+-----------+------------------+------------------+
|     Tablet| 66206.01999999997|28.979999999999972|
| Headphones| 65560.91000000003|32.880000000000024|
| Smartwatc

(None, None)

In [64]:
promotion_impact_on_month = with_disc.groupBy('month').agg(sum('price').alias('Total_sales'),sum('tax_rate').alias('Total_tax')).orderBy(col("month").asc())
no_promotion_impact_on_month = withtout_disc.groupBy('month').agg(sum('price').alias('Total_sales'),sum('tax_rate').alias('Total_tax')).orderBy(col("month").asc())

promotion_impact_on_month.show(),no_promotion_impact_on_month.show()


+-----+-----------------+------------------+
|month|      Total_sales|         Total_tax|
+-----+-----------------+------------------+
|    1|91432.65000000007| 44.33000000000005|
|    2|76862.18999999997| 38.55000000000002|
|    3|87236.74000000002| 42.60000000000011|
|    4|90333.43000000002|45.230000000000096|
|    5|92913.06000000004| 45.76000000000007|
|    6|         84020.54| 39.53000000000004|
|    7|98695.13999999996| 46.79000000000012|
|    8|92365.96000000004|44.820000000000086|
|    9|96101.31000000001|  45.2400000000001|
|   10|95210.99999999997|  48.3500000000001|
|   11| 93078.7999999999|43.720000000000084|
|   12|86992.45000000011| 43.34000000000008|
+-----+-----------------+------------------+

+-----+------------------+------------------+
|month|       Total_sales|         Total_tax|
+-----+------------------+------------------+
|    1| 45878.08000000002| 20.44000000000001|
|    2|29958.009999999995| 14.23999999999999|
|    3| 38463.45000000002|20.069999999999997|
|  

(None, None)

In [65]:
promotion_impact_on_day = with_disc.groupBy('day_name').agg(sum('price').alias('Total_sales'),sum('tax_rate').alias('Total_tax')).orderBy(col("Total_sales").desc(), col("Total_tax"))
no_promotion_impact_on_day = withtout_disc.groupBy('day_name').agg(sum('price').alias('Total_sales'),sum('tax_rate').alias('Total_tax')).orderBy(col("Total_sales").desc(), col("Total_tax"))

promotion_impact_on_day.show(),no_promotion_impact_on_day.show()


+---------+------------------+-----------------+
| day_name|       Total_sales|        Total_tax|
+---------+------------------+-----------------+
| Saturday|169794.43000000002| 80.7000000000001|
| Thursday|162718.23999999993|            77.69|
|  Tuesday|         161253.71|79.13000000000005|
|   Friday|159741.35999999993|80.85000000000008|
|   Sunday|154213.50999999992|            74.79|
|   Monday| 139239.9700000001| 69.0400000000001|
|Wednesday|138282.05000000005|66.06000000000022|
+---------+------------------+-----------------+

+---------+------------------+------------------+
| day_name|       Total_sales|         Total_tax|
+---------+------------------+------------------+
|  Tuesday|          77735.02| 36.35000000000001|
| Saturday| 72905.94999999994| 36.10000000000003|
| Thursday| 68383.89999999998| 34.32000000000002|
|   Sunday| 65847.15000000004|34.239999999999995|
|   Friday| 60488.18000000005|30.940000000000005|
|Wednesday|59057.780000000006|28.580000000000016|
|   Monday

(None, None)

In [67]:
##### 8) tax rate analyses ####
tax_rate_df = cust_trans_product_prom_df.select("tax_rate","price")


In [68]:
############## deviding the tax rates into three buckets for analysis ###################
from pyspark.sql.functions import expr
from pyspark.sql.types import StringType

# Calculate percentiles
percentiles = tax_rate_df.approxQuantile("tax_rate", [0.25, 0.5, 0.75], 0.01)

# Define a function to categorize values based on percentiles
def categorize_percentile(value):
    if value < percentiles[0]:
        return "very low"
    elif value < percentiles[1]:
        return "low"
    elif value < percentiles[2]:
        return "medium"
    else:
        return "high"

# Register the function as a UDF
categorize_percentile_udf = udf(categorize_percentile, StringType())

# Apply the UDF to create a new column with categories
tax_rate_df = tax_rate_df.withColumn("category", categorize_percentile_udf(tax_rate_df["tax_rate"]))

# Show the DataFrame with categories
# tax_rate_df.show()


In [69]:
###bucket wise tax analysis on sales
tax_rate_df_analysis = tax_rate_df.groupBy('category').agg(sum('price').alias('Total_sales'),sum('tax_rate').alias('Total_tax')).orderBy(col("Total_sales").desc(), col("Total_tax"))
tax_rate_df_analysis.show()

+--------+-----------------+------------------+
|category|      Total_sales|         Total_tax|
+--------+-----------------+------------------+
|    high|699221.7099999996| 473.4499999999781|
|  medium|434981.0899999999| 170.4999999999946|
|     low|414313.6500000002|112.27999999999554|
+--------+-----------------+------------------+



In [70]:
###### 9) & 10) coss-selling and upselling and marketbasket analysis
#### Apiori algorithm
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [71]:
from pyspark.sql.functions import lit, when, col, count

mba_df = cust_trans_product_prom_df.select('transaction_id','description')
mba_df = mba_df.withColumn("buy",lit(1))
mba_df = mba_df.groupby('transaction_id').pivot("description").agg(count(when(col("buy") >= 1, 1)))
mba_df = mba_df.fillna(0)
mba_df = mba_df.drop('transaction_id')
mba_df.show(truncate=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


+------+----------+--------+------+-------+-----+-------+----------+----------+------+
|Camera|Headphones|Keyboard|Laptop|Monitor|Mouse|Printer|Smartphone|Smartwatch|Tablet|
+------+----------+--------+------+-------+-----+-------+----------+----------+------+
|0     |0         |0       |0     |1      |0    |0      |0         |0         |0     |
|1     |0         |0       |1     |1      |0    |0      |0         |0         |0     |
|0     |0         |0       |0     |1      |0    |0      |0         |0         |1     |
|0     |0         |1       |0     |0      |1    |0      |0         |0         |0     |
|0     |0         |0       |0     |0      |0    |1      |0         |0         |0     |
|0     |0         |0       |0     |0      |0    |0      |0         |0         |1     |
|0     |0         |1       |0     |0      |0    |0      |0         |0         |0     |
|0     |0         |0       |0     |0      |1    |0      |0         |0         |1     |
|0     |1         |0       |0     |0      |

In [72]:
# Define a function to convert values greater than 1 to 1
def convert_to_one(value):
    return when(col(value) > 1, 1).otherwise(col(value))

# Apply the function to all numeric columns in the DataFrame
for col_name in mba_df.columns:
    if col_name != 'transaction_id':  # Skip non-numeric columns
        mba_df = mba_df.withColumn(col_name, convert_to_one(col_name))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [73]:
"#applying the apriori algo"
frequent_itemsets = apriori(mba_df.toPandas(), min_support=0.2, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/usr/local/lib/python3.10/dist-packages/pyspark/sql/pandas/utils.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(pandas.__version__) < LooseVersion(minimum_pandas_version):
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computati

In [74]:
frequent_itemsets

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
0,0.208,(Headphones)
1,0.204,(Smartwatch)
2,0.217,(Tablet)


In [75]:
rules[['antecedents','consequents','confidence']]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,confidence


#More possible Analysis

In [77]:
# 14) we can also find the similer user purchase behaiviour using collarative filternig ML approach based on that we can recommend product to users so that it will help in increase sales.
# 15) We can also analyse data based on the GENDER categories so that we can target perticuler product more on interested GENDER group.
# 16) We can also analyse data based on the AGE and Geolocation as bivariate analsis, so that we can target perticuler product more on interested group.
# 17) Based on the membership analysis we can analyse which segment purchasing more items based on that we can aquire more memberships in effective manner.
# 18) based on the product_name, discount and price column we can analyse the relationship of sales and discount, based on that we can decide effective discount on product for more sales.

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#important do not delete
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import explode, udf
# from pyspark.sql.types import ArrayType, StringType
# from ast import literal_eval
# # Define a UDF to convert the string representation of the list to an actual list of dictionaries
# def convert_to_list(s):
#     return literal_eval(s)
# exploded_df = transaction_df.withColumn("exploded", explode(udf(lambda x: literal_eval(x), ArrayType(StringType()))(transaction_df["items"]))).show(truncate=False)
